In [7]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(os.getcwd(), '..', '..'))

In [11]:
%matplotlib inline

from session.sessions import selected_009266
from analysis.imports import *
#from analysis.loading import load_session_data

In [12]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

import h5py
import shutil
import json
import random

from scipy import stats
from datetime import datetime
from functools import reduce

import nbimporter
from utils import get_sessions_list

In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
def get_channels_number(xml_path):
    if not os.path.exists(xml_path):
        return None
    root = ET.parse(xml_path).getroot()
    n_channels = root.findall('acquisitionSystem')[0].findall('nChannels')[0]
    return int(n_channels.text)

In [12]:
def check_session(path):
    animal = os.path.basename(os.path.normpath(path)).split('_')[0]
    filebase = os.path.basename(path)
    #has_contours = lambda x: len([f for f in os.listdir(path) if f == 'contours.csv' ]) > 0
    #has_h5_track = lambda x: len([f for f in os.listdir(path) if f.startswith(animal) and f.endswith('.h5')]) > 0
    has_h5_ephys = lambda x: len([f for f in os.listdir(path) if f.startswith('Andrey') and f.endswith('.h5')]) > 0
    has_dat = lambda x: len([f for f in os.listdir(path) if f.startswith(animal) and f.endswith('.dat')]) > 0
    has_clu = lambda x: len([f for f in os.listdir(path) if f.startswith(animal) and f.find('.clu') > 0 and f.find('bak') < 0])
    ch_num  = lambda x: get_channels_number(os.path.join(path, '%s.xml' % filebase))
    
    return [has_h5_ephys(path), has_dat(path), has_clu(path), ch_num(path)]

## Unit count

In [14]:
sessions = [s for s in selected_009266.keys()]
sessions.sort()
sessions

['009266_hippoSIT_2023-04-13_08-57-46',
 '009266_hippoSIT_2023-04-14_09-17-34',
 '009266_hippoSIT_2023-04-17_09-06-10',
 '009266_hippoSIT_2023-04-17_17-04-17',
 '009266_hippoSIT_2023-04-18_10-10-37',
 '009266_hippoSIT_2023-04-18_17-03-10',
 '009266_hippoSIT_2023-04-19_10-33-51',
 '009266_hippoSIT_2023-04-20_08-57-39',
 '009266_hippoSIT_2023-04-20_15-24-14',
 '009266_hippoSIT_2023-04-21_08-43-00',
 '009266_hippoSIT_2023-04-21_13-12-31',
 '009266_hippoSIT_2023-04-24_10-08-11',
 '009266_hippoSIT_2023-04-24_16-56-55',
 '009266_hippoSIT_2023-04-26_08-20-17',
 '009266_hippoSIT_2023-05-02_12-22-14',
 '009266_hippoSIT_2023-05-04_09-11-06',
 '009266_hippoSIT_2023-05-04_19-47-15',
 '009266_hippoSIT_2023-05-22_09-27-22',
 '009266_hippoSIT_2023-05-23_09-18-05',
 '009266_hippoSIT_2023-05-25_15-55-57',
 '009266_hippoSIT_2023-06-14_08-21-23',
 '009266_hippoSIT_2023-06-19_08-58-35']

In [17]:
unit_mx = np.zeros([len(sessions), 6])  # 6 shanks total
for i, session in enumerate(sessions):
    animal      = session.split('_')[0]
    h5_file     = os.path.join(source, animal, session, session + '.h5')
    
    with h5py.File(h5_file, 'r') as f:
        unit_names = [x for x in f['units']]
        
    shanks = np.unique([int(x[0]) for x in unit_names])
    shanks.sort()
    for shank in shanks:
        unit_mx[i, shank - 1] = len([x for x in unit_names if int(x[0]) == shank])

In [19]:
unit_mx.astype(np.int16)

array([[ 6,  2,  6,  3,  0,  0],
       [11,  1,  6,  2,  1,  0],
       [ 8,  1,  6,  5,  1,  0],
       [14,  2,  7,  3,  1,  0],
       [12, 13, 14,  6,  4,  0],
       [17, 10, 13,  5,  3,  0],
       [19,  5,  6,  8,  3,  0],
       [ 0,  4, 15,  8,  5,  3],
       [ 0,  1,  6,  6,  8,  3],
       [ 0,  4, 17,  8,  6,  5],
       [ 1,  4, 17,  8,  0,  1],
       [22, 21,  0,  9,  1,  3],
       [18,  1, 22,  4,  0,  7],
       [ 2, 24,  9,  6,  5,  7],
       [ 0, 17,  0,  9,  5,  8],
       [25, 29, 30, 14,  7, 19],
       [30, 14,  7,  7,  1,  3],
       [31, 34, 10,  2,  2, 12],
       [22, 43,  4,  0,  1,  6],
       [14, 41, 55,  2,  1,  7],
       [11,  0, 35,  1,  1,  2],
       [32,  0, 17,  1,  0,  0]], dtype=int16)

## Print some infos

In [5]:
source = '/home/sobolev/nevermind/Andrey/data'
animal = '009266'

In [8]:
s_list = get_sessions_list(os.path.join(source, animal), animal)

for session in s_list[:]:
    h5_file     = os.path.join(source, animal, session, session + '.h5')
    if not os.path.exists(h5_file):
        continue
        
    with h5py.File(h5_file, 'r') as f:
        #tl = np.array(f['processed']['timeline'])  # time, X, Y, speed, etc.
        #trials = np.array(f['processed']['trial_idxs'])  # t_start_idx, t_end_idx, x_tgt, y_tgt, r_tgt, result
        if not 'processed' in f:
            continue
        cfg = json.loads(f['processed'].attrs['parameters'])
        
    tgt = cfg['sound']['sounds']['target']
    bgr = cfg['sound']['sounds']['background']
    print("%s: Tgt: %.2f, %.2f, %.2f; Bgr: %.2f, %.2f, %.2f" % (session[-20:],\
            tgt['freq'], tgt['amp'], tgt['duration'],\
            bgr['freq'], bgr['amp'], bgr['duration'] \
    ))
    

_2022-12-20_16-33-45: Tgt: 660.00, 0.10, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-21_16-59-56: Tgt: 660.00, 0.10, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-22_10-27-35: Tgt: 660.00, 0.10, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-23_10-10-41: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-26_12-21-18: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-27_11-29-41: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.00, 0.05
_2022-12-27_17-22-28: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.05, 0.05
_2022-12-28_11-51-50: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.05, 0.05
_2022-12-28_15-51-10: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.05, 0.05
_2022-12-29_11-52-42: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.05, 0.05
_2023-02-02_17-05-17: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.05, 0.05
_2023-02-27_09-33-24: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.15, 0.05
_2023-02-28_10-11-03: Tgt: 660.00, 0.15, 0.05; Bgr: 660.00, 0.20, 0.05
_2023-02-28_19-53-56: Tgt: 660.00, 0.20, 0.05; Bgr: 660.00, 0.15, 0.05
_2023-

## AEPs check

In [24]:
def check_AEPs_file(sessionpath):
    # AEPs exists, aeps_events, A1 aeps, PPC aeps
    result = [False, False, False, False, False]
    
    aeps_file   = os.path.join(sessionpath, 'AEPs.h5')
    has_aeps = os.path.exists(aeps_file)
    if not has_aeps:
        return result
    
    result[0] = has_aeps
    
    with h5py.File(aeps_file, 'r') as f:
        result[1] = 'aeps_events' in f
        result[2] = 'A1' in f and 'aeps' in f['A1']
        result[3] = 'PPC' in f and 'aeps' in f['PPC']
        result[4] = 'HPC' in f and 'aeps' in f['HPC']
    
    return result

In [25]:
source = '/home/sobolev/nevermind/Andrey/data'
animals = ['009266']

In [26]:
s_list = reduce(lambda a, b: a+b, [get_sessions_list(os.path.join(source, animal), animal) for animal in animals])

checks = pd.DataFrame([[x] + check_AEPs_file(os.path.join(source, x.split('_')[0], x)) for x in s_list])
checks.columns = ['session', 'AEPs', 'events', 'A1', 'PPC', 'HPC']

with pd.option_context('display.max_rows', None,):
    print(checks)

                                session   AEPs  events     A1    PPC    HPC
0   009266_hippoSIT_2022-12-20_16-33-45  False   False  False  False  False
1   009266_hippoSIT_2022-12-21_16-59-56  False   False  False  False  False
2   009266_hippoSIT_2022-12-22_10-27-35  False   False  False  False  False
3   009266_hippoSIT_2022-12-23_10-10-41  False   False  False  False  False
4   009266_hippoSIT_2022-12-26_12-18-22  False   False  False  False  False
5   009266_hippoSIT_2022-12-26_12-21-18  False   False  False  False  False
6   009266_hippoSIT_2022-12-27_11-29-41  False   False  False  False  False
7   009266_hippoSIT_2022-12-27_17-22-28  False   False  False  False  False
8   009266_hippoSIT_2022-12-28_11-51-50  False   False  False  False  False
9   009266_hippoSIT_2022-12-28_15-51-10  False   False  False  False  False
10  009266_hippoSIT_2022-12-29_11-52-42  False   False  False  False  False
11  009266_hippoSIT_2023-02-02_17-05-17  False   False  False  False  False
12  009266_h

## Session health check

In [19]:
source = '/home/sobolev/nevermind/Andrey/data'
animals = ['009266']

In [20]:
# sessions check-up
s_list = reduce(lambda a, b: a+b, [get_sessions_list(os.path.join(source, animal), animal) for animal in animals])

checks = pd.DataFrame([[x] + check_session(os.path.join(source, x.split('_')[0], x)) for x in s_list])
checks.columns = ['session', 'contours', '.dat', '.clu', 'ch#']
#checks = checks.astype({"ch#": int})

def _color_red_or_green(val):
    if type(val) == type(True):
        color = 'green' if val else 'red'
        return 'color: %s' % color
    return 'color: black'

checks.style.applymap(_color_red_or_green)
#checks.head().style.applymap(_color_red_or_green)

,session,contours,.dat,.clu,ch#
0,009266_hippoSIT_2023-02-28_19-53-56,False,True,6,64.000000
1,009266_hippoSIT_2023-03-01_17-53-11,False,True,6,64.000000
2,009266_hippoSIT_2023-03-06_15-10-36,False,True,6,64.000000
3,009266_hippoSIT_2023-03-06_20-43-19,False,True,6,64.000000
4,009266_hippoSIT_2023-03-08_17-06-45,False,True,6,64.000000
5,009266_hippoSIT_2023-03-09_09-37-07,False,True,6,64.000000
6,009266_hippoSIT_2023-03-09_19-12-22,False,True,6,64.000000
7,009266_hippoSIT_2023-04-12_15-49-49,False,False,0,nan
8,009266_hippoSIT_2023-04-13_08-57-46,False,True,6,64.000000
9,009266_hippoSIT_2023-04-14_09-17-34,False,True,6,64.000000


In [1]:
#[x for x in checks['session']]

In [ ]:
"""
008228

Training between 2022-03-08_11-14-35 and 2022-04-01_09-53-55

Recording (swapped channels on 2) between 2022-05-02_10-32-35 and 2022-05-23_20-07-44 - investigate where are electrodes

Recording between 2022-05-27_10-24-45 and 2022-06-03_12-13-09 good signal, CA1

Training between 2022-06-07_15-28-17 and


008228_hippoSIT_2022-05-31_10-10-19 - 2, 3, 4, 5
the rest - 2, 3, 4, 5, 7, 8

"""

In [ ]:
"""
008229

Training between 2022-03-08_11-55-41 and 2022-05-05_10-08-48

Recording (swapped channels on 2) between 2022-05-09_15-55-58 and 2022-05-23_21-10-47

Recording between 2022-05-25_13-03-24 and 2022-06-03_09-18-49 - Need to make a correct XML (!)

Training between 2022-06-07_15-28-17 and

--- redo ss these:
008229_hippoSIT_2022-06-03_09-18-49 - 1,       4, 5, 6,   (8)
008229_hippoSIT_2022-06-02_20-42-08 - 1,       4, 5, 6,    8
008229_hippoSIT_2022-06-02_15-26-01 - 1,       4, 5, 6,    8
008229_hippoSIT_2022-06-01_16-09-26 - 1,       4, 5, 6,    8
008229_hippoSIT_2022-06-01_11-08-51 - 1,       4, 5, 6,    8
008229_hippoSIT_2022-05-31_16-38-23 - 1,       4, 5, 6,    8
008229_hippoSIT_2022-05-31_11-00-02 - 1,       4, 5,    7, 8
008229_hippoSIT_2022-05-30_21-40-24 - 1,       4, 5, 6,    8

008229_hippoSIT_2022-05-30_15-35-25 -             5, 6, 7, 8 
008229_hippoSIT_2022-05-29_17-16-59 -             5,    7, 8 (not much here, but some cells are good)
008229_hippoSIT_2022-05-29_10-45-44 -             5,    7, 8 (not much here)
008229_hippoSIT_2022-05-28_13-21-39 -             5, 6, 7, 8
008229_hippoSIT_2022-05-27_19-47-51 -             5, 6, 7, 8 
008229_hippoSIT_2022-05-27_14-21-37 -             5,    7, 8 (many broken data chunks)

---- CA1 starts here

008229_hippoSIT_2022-05-26_16-21-45 -       3, 4,         8 (not much here)
008229_hippoSIT_2022-05-25_13-03-24 -    2, 3, 4,         8 (not much here)

----
Here is the point where I changed the channel mapping on OEphys, so
corresponding .xml config file is changed too
----

008229_hippoSIT_2022-05-23_21-10-47 - 1, 2, 3, 4, 5, 6
008229_hippoSIT_2022-05-23_15-45-49 - 1, 2, 3, 4, 5, 6, 7
008229_hippoSIT_2022-05-20_15-54-39 - 1, 2, 3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-18_14-36-18 - 1,    3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-17_21-44-43 - 1,    3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-17_15-42-49 - 1,    3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-16_20-36-44 - 1, 2,    4, 5, 6, 7, 8
008229_hippoSIT_2022-05-16_11-13-33 - 1, 2, 3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-13_15-17-12 - 1, 2, 3, 4, 5, 6, 7, 8
008229_hippoSIT_2022-05-13_09-48-00 - 1, 2,    4, 5, 6, 7, 8
008229_hippoSIT_2022-05-12_15-36-57 - 1,       4, 5, 6, 7, 8
008229_hippoSIT_2022-05-11_13-55-55 - 1,       4, 5, 6, 7, 8
008229_hippoSIT_2022-05-10_14-01-46 - 1, 2,       5, 6,    8
008229_hippoSIT_2022-05-10_09-02-35 - 1, 2,       5, 6,    8
008229_hippoSIT_2022-05-09_21-01-15 - 1, 2,    4, 5, 6,    8
008229_hippoSIT_2022-05-09_15-55-58 - 1, 2,       5, 6,    8


COLORS
blue - 07a0c3
yellow - f0c808
white - fff1d0
red - dd1c1a


"""

In [32]:
import shutil

# some file management
source = '/home/sobolev/nevermind/Andrey/data'
animal = '008228'
sessions = [
"008228_hippoSIT_2022-05-02_10-32-35",
"008228_hippoSIT_2022-05-02_15-55-05",
"008228_hippoSIT_2022-05-03_09-00-15",
"008228_hippoSIT_2022-05-03_16-22-57",
"008228_hippoSIT_2022-05-04_09-17-49",
"008228_hippoSIT_2022-05-04_20-48-50",
"008228_hippoSIT_2022-05-05_09-22-16",
"008228_hippoSIT_2022-05-11_15-00-20",
"008228_hippoSIT_2022-05-12_14-47-34",
"008228_hippoSIT_2022-05-13_10-44-16",
"008228_hippoSIT_2022-05-13_15-59-43",
"008228_hippoSIT_2022-05-16_10-11-01",
"008228_hippoSIT_2022-05-16_19-50-53",
"008228_hippoSIT_2022-05-17_17-10-14",
"008228_hippoSIT_2022-05-19_09-31-53",
"008228_hippoSIT_2022-05-20_15-11-23",
"008228_hippoSIT_2022-05-23_14-10-04",
"008228_hippoSIT_2022-05-23_20-07-44",
]
f_types = ['clu', 'fet', 'klg', 'res', 'spk']
electrodes = [1, 2, 3, 4, 5, 6, 7, 8]

for session in sessions:
    
    # delete files
    s_path = os.path.join(source, animal, session)
    combs = ["%s.%s" % (f_type, electrode) for f_type in f_types for electrode in electrodes]
    
    to_delete = filter(lambda x: len([j for j in combs if x.find(j) > 0]) > 0, os.listdir(s_path))
    to_delete = [f for f in to_delete if os.path.exists(os.path.join(source, animal, session, f))]
    
    for f in to_delete:
        os.remove(os.path.join(source, animal, session, f))
        
    print('Removed %s files' % len(to_delete))
    
    # copy XML file
    xml_src = os.path.join(source, animal, 'settings', 'ns_settings_before_2022-05-25.xml')
    xml_dst = os.path.join(s_path, '%s.xml' % session)
    
    shutil.copyfile(xml_src, xml_dst)

Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 0 files
Removed 27 files
Removed 30 files
Removed 0 files
Removed 36 files
Removed 0 files
